# Comparison - Exercise

For this exercise, we will expand our previous fit to include the measurements of a conductivity sensor. We will also include the replicate that our colleagues in the lab made to control for system variance. You can find all required data in the labbook.

## Exercise 1: Comparator

The experiment was repeated to account for system variability. The data was converted to concentrations in $mM$ and can be found in `./experimental_data`.

**Task:**
- Import and plot the experimental data using the `ReferenceIO` class.
- Add the references to the `Comparator`.
- Add the `SSE` difference metric and compare with simulation results.
- Compare with other metrics.
- *Consider the accuracy of the model fit and the model complexity. Are there details in the data that your model is not yet capturing?*

```{note}
It's also possible to add multiple references, e.g. for triplicate experiments or for different sensors.
```